In [ ]:
#Importing the necessary libraries

import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import torch
import numpy as np
import pickle
import open3d as o3d
import matplotlib.pyplot as plt
from model.resunet import ResUNetBN2C
from util.visualization import get_colored_point_cloud_feature
from numpy import linalg as LA
from util.feature_extraction import stl2ply_convert, feature_extract
from util.misc import sort_list, zero_pad
from util.spp_layer import  SpatialPyramidPooling
from collections import Counter
from tqdm import tqdm
from tensorflow.python.keras.layers import Layer
import tensorflow.keras.backend as K
K.image_data_format()

In [ ]:
# Unpickling the features previously extracted

with open("data/names_paths_features_featurenet.dat", "rb") as f:
    file_names, file_paths, features = pickle.load(f)

In [ ]:
# Checking the number of classes and the data count
families = Counter(file_names)
print("Number of classes:",len(families))
print("\n",families)

In [ ]:
# Visualizing the distrubtion of feaure dimensions along all the 24 classes
shape_list = list()
for i in range(0,len(features)):
    f = features[i]
    shape_list.append(f.shape[0])
xs = [x for x in range(len(shape_list))]
plt.plot(xs,shape_list)

In [ ]:
# Checking maximum feature dimension
max_val=0

for i in range(0,len(features)):
    f = features[i]
    max = np.maximum(f.shape[0],f.shape[1])
    if(max > max_val):
        max_val = max
print(max_val)

In [ ]:
# Zero padding all the vectors to max value
def zero_pad(x):
    shape_pad = max_val 
    x1 = np.zeros([shape_pad,32])
    x1[:x.shape[0],:x.shape[1]] = x
    return x1
max_val = 240
for j in range(0,len(features)):
    z = zero_pad(features[j])
    features[j] =z

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical

X = np.asarray(features)

# one-hot encoding
Y_set = set(file_names)
Y_list = (list(Y_set))
mapping = {}
family = list()

# 24 unique features
for x in range(len(Y_list)):
  mapping[Y_list[x]] = x

# mapping to corresponding category
for x in range(len(file_names)):
  family.append(mapping[file_names[x]])

Y = to_categorical(family)
Y = np.array(Y)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=1)
# val,test = 0.5 x 0.3 = 0.15

In [ ]:
# Convering to tensors
x_train = tf.convert_to_tensor(x_train, dtype=None, dtype_hint=None, name=None)
x_val = tf.convert_to_tensor(x_val, dtype=None, dtype_hint=None, name=None)
x_test = tf.convert_to_tensor(x_test, dtype=None, dtype_hint=None, name=None)
y_train = tf.convert_to_tensor(y_train, dtype=None, dtype_hint=None, name=None)
y_val = tf.convert_to_tensor(y_val, dtype=None, dtype_hint=None, name=None)
y_test = tf.convert_to_tensor(y_test, dtype=None, dtype_hint=None, name=None)

In [ ]:
# Reshaping tensor for input to neural net

x_train = tf.reshape(x_train, [len(x_train),max_val,32,1])
x_val = tf.reshape(x_val, [len(x_val),max_val,32,1])
x_test = tf.reshape(x_test, [len(x_test),max_val,32,1])

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, Activation, MaxPooling2D, Dense, Dropout


# Minimizes Tensorflow Logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

BATCH_SIZE = 64
NUM_CHANNELS = 1
NUM_CLASSES = 24

def makeModel():
    model = Sequential()

    # MODEL 1
    # uses tensorflow ordering. Note that we leave the image size as None to allow multiple image sizes
    model.add(Convolution2D(32, 3, NUM_CHANNELS, padding='same', input_shape=(max_val,32,1)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, NUM_CHANNELS, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Convolution2D(64, 3, NUM_CHANNELS, padding='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, NUM_CHANNELS, padding='same'))
    model.add(Activation('relu'))
    model.add(SpatialPyramidPooling([1, 2, 4]))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))

    return model

In [ ]:
tf.executing_eagerly = False

In [ ]:
model=makeModel()
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
# Running and Saving model

from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "model/pretrained/spp_featurenet.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(x_train, y_train, batch_size=1, epochs= 30, verbose=2,
                    validation_data=(x_val,y_val), shuffle=True, class_weight=None,
                    sample_weight=None, initial_epoch=0, steps_per_epoch=None, 
                    validation_steps=None, validation_freq=1, max_queue_size=10,
                    workers=1,callbacks=callbacks_list)

In [ ]:
#loading the model

from  tensorflow.keras.models import load_model
model = load_model("model/pretrained/spp_featurenet.h5",custom_objects={'SpatialPyramidPooling': SpatialPyramidPooling})

In [ ]:
# Defining a temperory model for spp output

from  tensorflow.keras.models import Model
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer('spatial_pyramid_pooling').output)

## Finding Accuracy

In [ ]:
# Finding accuracy on test set
spp_train = intermediate_layer_model.predict(x_train)

top5_acc =0
top1_acc =0
top5_lbl = list()
sim_list = list()
null_index = list()
classes = 24

for i in tqdm (range(0,len(x_test))):
    sim_list.clear()
    #test_feat = spp_test[i]
    test_feat = tf.reshape(x_test[i],[1,max_val,32,1])
    spp_test = intermediate_layer_model.predict(test_feat)
    y_t = y_test[i]
    for j in range(0,classes):
        if(y_t[j].numpy()==1.0):
            test_lbl = Y_list[j]  
    for k in range(0,len(x_train)):  
        #train_feat = tf.reshape(x_train[i],[1,max_val,32,1])
        sim_list.append(abs(np.linalg.norm(spp_train[k]-spp_test)))
    id = list(range(0,len(x_train)))
    Sim_models_id = [x for _,x in sorted(zip(sim_list,id))]
    top5 = (Sim_models_id[0:5])
    top5_lbl.clear()
    for l in range(0,5): 
        yi = y_train[top5[l]]
        for m in range(0,classes):
            if ((yi[m].numpy())==1.0):
                top5_lbl.append(Y_list[m])
    if(test_lbl in top5_lbl):
        top5_acc+=1
        
    if (top5_lbl):
        if(test_lbl == top5_lbl[0]):
            top1_acc+=1
    else:
        null_index.append(i)
        
print("Accuracy for ",len(x_test),"testing files is ",top1_acc/len(x_test))
print("Top 5 accuracy for ",len(x_test),"testing files is ",top5_acc/len(x_test))
print(len(null_index))

## Retrieving similar features from the dataset for a sample file

In [ ]:
#loading stl dataset paths 

db_folder = "data/stl"
os.path.abspath(db_folder)
ind = 0
stl_file_path = list()
sub_folders = os.listdir(db_folder)
for sub_folder in sub_folders:
        sub_folder_path = os.path.join(db_folder, sub_folder)
        stl_files = os.listdir(sub_folder_path)
        for stl_file in stl_files:
            if stl_file.endswith(".STL"):
                stl_file_path.append(os.path.join(sub_folder_path, stl_file))
                ind+=1

In [ ]:
def get_spp_out(feat):
    test_feat = tf.reshape(feat,[1,max_val,32,1])
    spp_test = intermediate_layer_model.predict(test_feat)
    return spp_test    

In [ ]:
#test file

test_id = 1990
print("Test file\n","\nID:\t",test_id,"\tFamily:\t",file_names[test_id])
test_feat = zero_pad(features[test_id])
spp_test_feat = get_spp_out(test_feat)

In [ ]:
#comparing similaity between one test file and all the features individually

n_files = len(features)

sim_list = list()
for i in range(0,n_files):
    if(i!=test_id):
        feat = zero_pad(features[i])
        feat = tf.reshape(feat,[1,max_val,32,1])
        spp_feat = intermediate_layer_model.predict(feat)
        sim_list.append(abs(np.linalg.norm(spp_feat - spp_test_feat)))
    else:
        sim_list.append(float("inf"))

In [ ]:
id = list(range(0,n_files))

Similar_models_id = [x for _,x in sorted(zip(sim_list,id))]
top_5 = (Similar_models_id[0:5])
top_5

In [ ]:
# Visualiztion of the CAD files
from solid import*
import viewscad
r = viewscad.Renderer()
print("Test file\n","\nID:\t",test_id,"\tFamily:\t",file_names[test_id])
r.render_stl(stl_file_path[test_id])
print("\nTop-5 similar models and their IDs\n")
for i in range(0,5): 
    yi = file_names[top_5[i]]
    print("ID:\t",top_5[i],"\tFamily:\t",yi)
    r.render_stl(stl_file_path[top_5[i]])